In [4]:

table_product = """CREATE TABLE Product (
                      part TEXT PRIMARY KEY, 
                      part_name TEXT NOT NULL,
                      weight REAL
                      )"""

table_cost = """CREATE TABLE Cost (
                  part TEXT,
                  cost_date DATETIME NOT NULL,
                  cost REAl NOT NULL,
                  FOREIGN KEY (part) REFERENCES Product (part)
                  )"""

table_stock = """CREATE TABLE Stock (
                  part TEXT,
                  operation_date DATETIME NOT NULL,
                  qty INTEGER NOT NULL,
                  cost REAL  NOT NULL,
                  operation_type TEXT NOT NULL,
                  FOREIGN KEY (part) REFERENCES Product (part)
                  )"""

# створюємо таблиці в БД
import sqlite3
conn = sqlite3.connect('store.db')

curs = conn.cursor()

curs.execute(table_product)
curs.execute(table_cost)
curs.execute(table_stock)

conn.close()

In [5]:
# вносимо інформацію в таблицю Prodcut

conn = sqlite3.connect('store.db')
curs = conn.cursor()
sql_text = """INSERT INTO Product VALUES(?, ?, ?)"""

part = input('Введіть артикул товару')
part_name = input('Введіть назву товару')
weight = float(input('Введіть вагу товару'))

curs.execute(sql_text, (part, part_name, weight))
conn.commit()

conn.close()

Введіть артикул товаруre509672
Введіть назву товаруfilter
Введіть вагу товару300


In [6]:
conn = sqlite3.connect('store.db')
curs = conn.cursor()

sql_text = "SELECT * FROM Product"

curs.execute(sql_text)

for el in curs.fetchall():
    print(el)


conn.close()

('re509672', 'filter', 300.0)


In [7]:
#записуємо ціну
import datetime

conn = sqlite3.connect('store.db')
curs = conn.cursor()
sql_text = """INSERT INTO Cost VALUES(?, ?, ?)"""

part = input('Введіть артикул товару')
date = datetime.datetime.now()
cost = float(input('Введіть ціну товару'))

curs.execute(sql_text, (part, date, cost))
conn.commit()

conn.close()

Введіть артикул товаруre509672
Введіть ціну товару400


In [8]:
#записуємо розхід

import datetime

conn = sqlite3.connect('store.db')
curs = conn.cursor()


part = input('Введіть артикул товару')
date = datetime.datetime.now()
qty = int(input('Введіть кількість товару'))
operation_type = input('Введіть тип операції: 1-прихід || 2-розхід')

curs.execute("SELECT cost, cost_date FROM Cost WHERE part = (?) ORDER BY cost_date ", (part,) )

cost = curs.fetchall()[-1][0]


sql_text = """INSERT INTO Stock VALUES(?, ?, ?, ?, ?)"""
curs.execute(sql_text, (part, date, qty, cost, operation_type))
conn.commit()

conn.close()

Введіть артикул товаруre509672
Введіть кількість товару1
Введіть тип операції: 1-прихід || 2-розхід2


In [9]:
# почали писати на уроці і не закінчили. варіант рішення нижче
# import requests 
# from bs4 import BeautifulSoup
# r = requests.get('https://smallpacking.agrosem.ua/shop/')
# soup = BeautifulSoup(r.text, 'html.parser')

# all_products = {}

# def get_soup(href):
#     r = requests.get(href)
#     soup = BeautifulSoup(r.text, 'html.parser')
#     return soup

# def product_informations(base, href):
#     p = get_html_text(href)
#     sku = p.find('span', class_='sku')
#     print(sku)
    
    
# def all_products_page(page_text):
#     l = []
#     for el in soup.find_all('div', class_="product-from-category-container"):
#         l.append(el.find('a').get('href'))
#     return l
        
# def all_page(href):
#     p = get_html_text(href)
#     list_with_product = all_products_page(p)
#     print(list_with_product)
    
# all_page('https://smallpacking.agrosem.ua/shop/')

In [10]:
#записуємо прихід

import datetime

conn = sqlite3.connect('store.db')
curs = conn.cursor()


part = input('Введіть артикул товару')
date = datetime.datetime.now()
qty = int(input('Введіть кількість товару'))
operation_type = input('Введіть тип операції: 1-прихід || 2-розхід')

curs.execute("SELECT cost, cost_date FROM Cost WHERE part = (?) ORDER BY cost_date ", (part,) )

cost = curs.fetchall()[-1][0]


sql_text = """INSERT INTO Stock VALUES(?, ?, ?, ?, ?)"""
curs.execute(sql_text, (part, date, qty, cost, operation_type))
conn.commit()

conn.close()

Введіть артикул товаруre509672
Введіть кількість товару1
Введіть тип операції: 1-прихід || 2-розхід1


In [11]:
import requests 
from bs4 import BeautifulSoup
r = requests.get('https://smallpacking.agrosem.ua/shop/')
soup = BeautifulSoup(r.text, 'html.parser')

all_products = {}

# def get_soup(href):
#     r = requests.get(href)
#     soup = BeautifulSoup(r.text, 'html.parser')
#     return soup
for i in range(1, 4):
    r = requests.get(f'https://smallpacking.agrosem.ua/shop/page/{i}/')
    soup = BeautifulSoup(r.text, 'html.parser')
    for el in soup.find_all('div', class_="product-from-category-container"):
        href = el.find('a').get('href')
        r = requests.get(href)
        soup = BeautifulSoup(r.text, 'html.parser')
        sku = soup.find('span', class_='sku').text.strip('SKU: ')
        name = soup.find('div', class_="atributes-description").find('h2').text
        price = float(soup.find('span', class_='regular-price').text.rstrip(' грн'))
        p = soup.find('span', class_="type-packaging-name").text
        if p == 'Коробка':
            weight = 1
        elif p == 'Мішок':
            weight = 3
        elif p == 'Відро':
            weight = 10
        else:
            weight = None

        all_products[sku] = {'name':name, 'price':price, 'weight':weight}
        
        
all_products

{'797756': {'name': 'Добриво для винограду', 'price': 150.0, 'weight': 1},
 '8362626': {'name': 'Добриво для вічнозелених дерев та кущів',
  'price': 150.0,
  'weight': 1},
 '4738184': {'name': 'Добриво для вічнозелених дерев та кущів',
  'price': 360.0,
  'weight': 3},
 '5218': {'name': 'Добриво для вічнозелених дерев та кущів',
  'price': 995.0,
  'weight': 10},
 '7143174': {'name': 'Добриво для газону', 'price': 360.0, 'weight': 3},
 '375738': {'name': 'Добриво для газону', 'price': 995.0, 'weight': 10},
 '789868': {'name': 'Добриво для газону', 'price': 150.0, 'weight': 1},
 '78747': {'name': 'Добриво для декоративних листових',
  'price': 360.0,
  'weight': 3},
 '3593': {'name': 'Добриво для полуниці', 'price': 150.0, 'weight': 1},
 '33486': {'name': 'Добриво для декоративних листових',
  'price': 150.0,
  'weight': 1},
 '198742': {'name': 'Добриво для овочів', 'price': 360.0, 'weight': 3},
 '88255': {'name': 'Добриво для овочів', 'price': 150.0, 'weight': 1},
 '68183': {'name': '

In [12]:
import sqlite3 
conn = sqlite3.connect('store.db')
curs = conn.cursor()
curs.execute('DELETE FROM Product')

sql_text = """INSERT INTO Product VALUES(?, ?, ?)"""


for k,v in all_products.items():
    curs.execute(sql_text, (k, v['name'], v['weight']))
    conn.commit()

conn.close()

In [13]:
import sqlite3 
conn = sqlite3.connect('store.db')
curs = conn.cursor()
curs.execute('SELECT * FROM Product')

for el in curs.fetchall():
    print(el)

conn.close()

('797756', 'Добриво для винограду', 1.0)
('8362626', 'Добриво для вічнозелених дерев та кущів', 1.0)
('4738184', 'Добриво для вічнозелених дерев та кущів', 3.0)
('5218', 'Добриво для вічнозелених дерев та кущів', 10.0)
('7143174', 'Добриво для газону', 3.0)
('375738', 'Добриво для газону', 10.0)
('789868', 'Добриво для газону', 1.0)
('78747', 'Добриво для декоративних листових', 3.0)
('3593', 'Добриво для полуниці', 1.0)
('33486', 'Добриво для декоративних листових', 1.0)
('198742', 'Добриво для овочів', 3.0)
('88255', 'Добриво для овочів', 1.0)
('68183', 'Добриво для розсади', 1.0)
('51561', 'Добриво для тепличних овочів', 1.0)
('5656841', 'Добриво для тепличних овочів', 3.0)
('2217672', 'Добриво для троянд', 1.0)
('439226', 'Добриво для фруктових та горіхових дерев', 3.0)
('3177412', 'Добриво для фруктових та горіхових дерев', 1.0)
('1297', 'Добриво для ягідних чагарників', 3.0)
('91897', 'Добриво для ягідних чагарників', 1.0)
('61169', 'Добриво універсальне грунтове', 1.0)
('6651824

In [14]:
import sqlite3 
from datetime import datetime, timedelta
import random
conn = sqlite3.connect('store.db')
curs = conn.cursor()


sql_text = """INSERT INTO Cost VALUES(?, ?, ?)"""


for k,v in all_products.items():
    curs.execute(sql_text, (k, datetime.now() + timedelta(days=random.randrange(100)), v['price']))
    conn.commit()

conn.close()

In [ ]:
"""CREATE TABLE Stock (
                  part TEXT,
                  operation_date DATETIME NOT NULL,
                  qty INTEGER NOT NULL,
                  cost REAL  NOT NULL,
                  operation_type TEXT NOT NULL,
                  FOREIGN KEY (part) REFERENCES Product (part)
                  )"""

In [16]:
import sqlite3 
from datetime import datetime, timedelta
import random
conn = sqlite3.connect('store.db')
curs = conn.cursor()


sql_text = """INSERT INTO Stock VALUES(?, ?, ?, ?, ?)"""

def product_cost(part):
    curs.execute("SELECT cost, cost_date FROM Cost WHERE part = (?) ORDER BY cost_date ", (part,) )
    cost = curs.fetchall()[-1][0]
    return cost

for k in all_products:
    for i in range(random.randrange(10)):
        curs.execute(sql_text, (k 
                                ,datetime.now() + timedelta(days=random.randrange(50))
                               , random.randrange(10, 20)
                               , product_cost(k)
                               , 'Розхід'))
        conn.commit()

conn.close()

In [17]:
import sqlite3

def result(sql_text):
    conn = sqlite3.connect('store.db')
    curs = conn.cursor()
    curs.execute(sql_text)

    for row in curs.fetchall():
        print(row)

    conn.close()

In [18]:
# skripts:
# 1. Вивести всю таблицю (10 позицій)
#  2. Перейменувати стовбці 
#  3. Вивести собівартість всього складу
#  4. Вивести товар (фільтр) 
#  5. Вивести прихід 
#  6. Вивести прихід більше 10 шт
#  7. Вивести назви всі продуктів 
#  8. Вивести продукт, по якому був прихід максильний 
#  9. Вивести залишки товарів 
#  10. Вивести товари яких більше 10 
#  11. Вести суму всього приходу по товарах 
#  12. Вивести прихід по товару, якого найбільше на складі 
#  13. Вивести приходи по місяцях 
#  14. Вивести залишки товарів та вагу 
#  15. Порахувати вартість 1 кг товару по кожному товару 
#  16. Вивести залишки товарів, вага яких більше 5 кг 
#  17. Порахувати ефективність закупки відносно останньої ціни 
#  18. Порахувати оборотність складу 
#  19. Вивести вартість складу в доларах ( на сьогоднішню дату)

script_1 = """
SELECT *
FROM Stock
LIMIT 10
"""
result(script_1)

('re509672', '2020-12-15 15:50:42.258732', 1, 400.0, '2')
('re509672', '2020-12-15 15:51:03.090322', 1, 400.0, '1')
('797756', '2020-12-29 15:51:52.814457', 169, 150.0, 'Прихід')
('797756', '2020-12-31 15:51:52.821436', 135, 150.0, 'Прихід')
('8362626', '2021-01-23 15:51:52.825425', 106, 150.0, 'Прихід')
('8362626', '2021-01-02 15:51:52.829414', 135, 150.0, 'Прихід')
('8362626', '2021-01-06 15:51:52.834405', 193, 150.0, 'Прихід')
('8362626', '2021-01-10 15:51:52.839391', 199, 150.0, 'Прихід')
('8362626', '2021-01-17 15:51:52.845545', 156, 150.0, 'Прихід')
('8362626', '2020-12-24 15:51:52.850598', 160, 150.0, 'Прихід')


In [19]:

script_2 = """
SELECT Part as p, Qty as q
FROM STOCK
LIMIT 10
"""
result(script_2)

('re509672', 1)
('re509672', 1)
('797756', 169)
('797756', 135)
('8362626', 106)
('8362626', 135)
('8362626', 193)
('8362626', 199)
('8362626', 156)
('8362626', 160)


In [20]:
script_4 = """
SELECT *
FROM STOCK
WHERE Part = '8362626'
"""
result(script_4)

('8362626', '2021-01-23 15:51:52.825425', 106, 150.0, 'Прихід')
('8362626', '2021-01-02 15:51:52.829414', 135, 150.0, 'Прихід')
('8362626', '2021-01-06 15:51:52.834405', 193, 150.0, 'Прихід')
('8362626', '2021-01-10 15:51:52.839391', 199, 150.0, 'Прихід')
('8362626', '2021-01-17 15:51:52.845545', 156, 150.0, 'Прихід')
('8362626', '2020-12-24 15:51:52.850598', 160, 150.0, 'Прихід')
('8362626', '2021-01-08 15:51:52.854358', 148, 150.0, 'Прихід')


In [21]:
script_5 = """
SELECT *
FROM STOCK
WHERE operation_type = 'Прихід'
LIMIT 10
"""
result(script_5)

('797756', '2020-12-29 15:51:52.814457', 169, 150.0, 'Прихід')
('797756', '2020-12-31 15:51:52.821436', 135, 150.0, 'Прихід')
('8362626', '2021-01-23 15:51:52.825425', 106, 150.0, 'Прихід')
('8362626', '2021-01-02 15:51:52.829414', 135, 150.0, 'Прихід')
('8362626', '2021-01-06 15:51:52.834405', 193, 150.0, 'Прихід')
('8362626', '2021-01-10 15:51:52.839391', 199, 150.0, 'Прихід')
('8362626', '2021-01-17 15:51:52.845545', 156, 150.0, 'Прихід')
('8362626', '2020-12-24 15:51:52.850598', 160, 150.0, 'Прихід')
('8362626', '2021-01-08 15:51:52.854358', 148, 150.0, 'Прихід')
('4738184', '2021-01-28 15:51:52.860338', 105, 360.0, 'Прихід')


In [22]:
script_11 = """
SELECT SUM(cost*qty)
FROM STOCK
"""
result(script_11)

(9539260.0,)


In [22]:
script_7 = """
SELECT DISTINCT Part
FROM STOCK
"""
result(script_7)

('re509672',)
('797756',)
('8362626',)
('4738184',)
('5218',)
('7143174',)
('375738',)
('789868',)
('78747',)
('3593',)
('33486',)
('198742',)
('88255',)
('68183',)
('5656841',)
('2217672',)
('439226',)
('1297',)
('91897',)
('61169',)
('6651824',)
('5227876',)
('4165215',)
('682645',)
('7435',)
('9354141',)
('1955',)
('455799',)
('726319',)
('16762',)
('17477',)
('51561',)
('3177412',)
('32237',)
('991645',)


In [23]:
script_8 = """
SELECT *
FROM Stock
WHERE qty = (SELECT MAX(qty)
             FROM Stock
             WHERE operation_type = 'Прихід')
"""
result(script_8)

('8362626', '2021-01-10 15:51:52.839391', 199, 150.0, 'Прихід')
('375738', '2021-01-20 15:51:52.928197', 199, 995.0, 'Прихід')
('3177412', '2021-01-25 15:51:53.108715', 199, 150.0, 'Прихід')


In [24]:
script_9 = """
SELECT part, SUM(qty)
FROM 
    (SELECT part, SUM(qty) as qty
    FROM Stock
    WHERE operation_type = 'Прихід' and operation_date < '2021-01-01'
    GROUP BY part
    

    UNION 

    SELECT part, -SUM(qty)
    FROM Stock
    WHERE operation_type = 'Розхід' and  operation_date < '2021-01-01'
    GROUP BY part)
GROUP BY part
"""
result(script_9)

('1297', 673)
('16762', 309)
('17477', 130)
('1955', 512)
('198742', 340)
('3177412', 185)
('32237', -36)
('33486', 125)
('3593', -10)
('375738', 170)
('4165215', 354)
('439226', 821)
('455799', 276)
('4738184', 148)
('51561', 261)
('5218', 279)
('5227876', 351)
('5656841', -28)
('6651824', -63)
('68183', -19)
('682645', 562)
('7143174', 149)
('726319', 324)
('7435', 137)
('78747', -16)
('789868', -30)
('797756', 292)
('8362626', 160)
('88255', -16)
('91897', -14)
('9354141', 496)
('991645', -40)


In [25]:
script_9 = """
SELECT part, SUM(qty)
FROM 
    (SELECT part, SUM(qty) as qty
    FROM Stock
    WHERE operation_type = 'Прихід' 
    GROUP BY part
    

    UNION 

    SELECT part, -SUM(qty)
    FROM Stock
    WHERE operation_type = 'Розхід' 
    GROUP BY part)
    
GROUP BY part
HAVING SUM(qty) > 1000 
"""
result(script_9)

('1297', 1258)
('16762', 1340)
('1955', 1014)
('682645', 1212)
('726319', 1020)
('8362626', 1097)
('9354141', 1160)


In [26]:
script_12 = """
SELECT *
FROM Stock
WHERE operation_type = 'Прихід'  AND part =  (SELECT Part
    FROM 
        (SELECT Part, MAX(qty)
        FROM 
            (SELECT part, SUM(qty) as qty
            FROM 
                (SELECT part, SUM(qty) as qty
                FROM Stock
                WHERE operation_type = 'Прихід' 
                GROUP BY part


                UNION 

                SELECT part, -SUM(qty)
                FROM Stock
                WHERE operation_type = 'Розхід' 
                GROUP BY part)

            GROUP BY part)))

"""
result(script_12)
# 1-й from вибрали залишки на складах
# 2-й from вибрали товар з максимальним залишком на складах
# 3-й from вибрали весьь прихід по товару, який має максимальний залишок

('16762', '2020-12-21 15:51:53.411158', 173, 995.0, 'Прихід')
('16762', '2021-01-23 15:51:53.415161', 144, 995.0, 'Прихід')
('16762', '2021-01-22 15:51:53.419132', 134, 995.0, 'Прихід')
('16762', '2020-12-27 15:51:53.423122', 136, 995.0, 'Прихід')
('16762', '2021-01-22 15:51:53.427111', 155, 995.0, 'Прихід')
('16762', '2021-01-16 15:51:53.431151', 179, 995.0, 'Прихід')
('16762', '2021-01-09 15:51:53.434143', 156, 995.0, 'Прихід')
('16762', '2021-01-17 15:51:53.438168', 134, 995.0, 'Прихід')
('16762', '2021-01-12 15:51:53.442121', 144, 995.0, 'Прихід')


In [27]:
script_13 = """
SELECT strftime('%Y', operation_date), strftime('%m', operation_date),  SUM(qty)
FROM Stock
WHERE operation_type = 'Розхід'
GROUP BY strftime('%Y', operation_date), strftime('%m', operation_date)
"""
result(script_11)

('2020', '12', 599)
('2021', '01', 1302)
('2021', '02', 157)


In [ ]:
script_14 = """
SELECT t1.part, SUM(t1.qty), t2.weight * SUM(t1.qty)
FROM 
    (SELECT part, SUM(qty) as qty
    FROM Stock
    WHERE operation_type = 'Прихід' 
    GROUP BY part
    

    UNION 

    SELECT part, -SUM(qty)
    FROM Stock
    WHERE operation_type = 'Розхід' 
    GROUP BY part) as t1
INNER JOIN Product as t2 ON t1.part = t2.part
    
GROUP BY t1.part
HAVING SUM(t1.qty) > 1000
"""
result(script_14)